# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

from src.data.dataset import make_dataset
from src.data.labels import make_labels
from src.models.HTK import hmm
from src.data.size_distribution import to_htk
from src.data.labels import read_dmps_maual_labels

from src import features
from src import models

from src.visualization import visualize as viz

from src.models.HTK import htktools as htkt

# Inicializo Verta

In [2]:
from verta import Client

HOST = "http://127.0.0.1:3000/"

PROJECT_NAME = "NPF Detector"
EXPERIMENT_NAME = "2_normalize_adapted"
EXPERIMENT_DESCRIPTION = "Data Version -> 2. Adapted model to real labels.Notebook -> 2-EventNoEvent_HInit_normalized_adapted.ipynb"


client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)
run = client.set_experiment_run(desc=EXPERIMENT_DESCRIPTION)

connection successfully established
set existing Project: NPF Detector from personal workspace
set existing Experiment: 2_normalize_adapted
created new ExperimentRun: Run 2163715995886778061295


# Preparo los datos

In [3]:
data_version = 2
normalize = True

train_data, test_data = make_dataset(normalize, data_version)

run.log_observation('number_of_test_files', test_data['count'])
run.log_observation('number_of_train_files', train_data['count'])

run.log_hyperparameters({'data_version' : data_version, 'normalize': normalize})

INFO:root:Converting raw files to HTK format ...
INFO:root:Data version: 2
INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:root:Generating script (.scp) files...
INFO:root:Adding deltas and acelerations...
INFO:root:Test files:	471
INFO:root:Train files:	4587


# Pongo las etiquetas

In [4]:
thresholds = {'nuc_threshold': 0.15,        # 1/cm3/10min
              'pos_vol_threshold': 200,     # 1/m3^3/10min
              'neg_vol_threshold': -5000    # 1/cm3/10min
              }  

labels = make_labels(thresholds, how='event-noevent', data_version=2)

run.log_hyperparameters(thresholds)

INFO:root:Creating labels ...
INFO:root:Labels created OK!
INFO:root:Generating Master Label File...


In [5]:
viz.generate_plots('synth_test_data', test_data, labels)

## Inicializo el modelo de Markov

Edito el el archivo hmmdefs y le agrego los modelos creados con HInit

In [6]:
vf = 0.1
mv = 0.1

model = hmm.initialize(train_data, variance_floor=vf, minimum_variance=mv)

run.log_hyperparameters({'var_floor': vf, 'var_min': mv})

INFO:root:Initializing model...


Calculating Fixed Variance
  HMM Prototype: ../src/models/HTK/misc/proto
  Segment Label: None
  Num Streams  : 1
  UpdatingMeans: Yes
  Target Direct: ../models/hmm/0
220176 speech frames accumulated
Updating HMM Means and Covariances
Output written to directory ../models/hmm/0
Var floor macros output to file ../models/hmm/0/vFloors



### Estimo los parametros y evaluo los resultados

In [7]:
model = hmm.train(model, train_data, labels)
results = hmm.test(model, test_data, labels)

for k,v in results.items():
    run.log_observation(k,v)

Pruning-Off

Pruning-Off

Pruning-Off

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C ../src/models/HTK/misc/config -H ../models/hmm/3/macros -H ../models/hmm/3/hmmdefs -p 0 -s 5 -A -S ../data/interim/test.scp -i ../data/interim/results.mlf -w ../src/models/HTK/misc/wdnet ../src/models/HTK/misc/dict ../src/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Tue Sep  8 15:16:11 2020
  Ref : ../data/interim/labels.mlf
  Rec : ../data/interim/results.mlf
------------------------ Overall Results --------------------------
SENT: %Correct=97.45 [H=153, S=4, N=157]
WORD: %Corr=96.02, Acc=96.02 [H=193, D=8, S=0, I=0, N=201]
------------------------ Confusion Matrix -------------------------
       e   n 
           e  Del [ %c / %e]
   e  37   0    2
  ne   0  156   6
Ins    0   0



In [8]:
viz.generate_plots('labeled_synth_test_data', test_data, labels, results)

# Aumento el numero de Gausianas

In [9]:
edit_commands = ['MU 2 {*.state[2-4].mix}']

model = hmm.edit(model, edit_commands)
model = hmm.train(model, train_data, labels)

results = hmm.test(model, test_data, labels)

for k,v in results.items():
    run.log_observation(k,v)

for command in edit_commands:    
    run.log_observation('model_edit_commands', command)

HHEd
 2/2 Models Loaded [5 states max, 1 mixes max]

MU 2 {}
 Mixup to 2 components per stream
 MU: Number of mixes increased from 6 to 12

Saving new HMM files ...
Edit Complete

Pruning-Off

Pruning-Off

Pruning-Off

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C ../src/models/HTK/misc/config -H ../models/hmm/7/macros -H ../models/hmm/7/hmmdefs -p 0 -s 5 -A -S ../data/interim/test.scp -i ../data/interim/results.mlf -w ../src/models/HTK/misc/wdnet ../src/models/HTK/misc/dict ../src/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Tue Sep  8 15:17:21 2020
  Ref : ../data/interim/labels.mlf
  Rec : ../data/interim/results.mlf
------------------------ Overall Results --------------------------
SENT: %Correct=96.82 [H=152, S=5, N=157]
WORD: %Corr=96.02, Acc=95.52 [H=193, D=8, S=0, I=1, N=201]
------------------------ Confusion Matrix -------------------------
       e   n 
           e  Del [ %c / %e]
   e  36

In [10]:
viz.generate_plots('labeled_synth_test_data_2_gauss', test_data, labels, results)

# Adapto el modelo a los datos reales

In [11]:
labeled_real_data, unlabeled_real_data = to_htk()
labels = read_dmps_maual_labels()

In [12]:
model = hmm.adapt(model, labeled_real_data, labels)
results = hmm.test(model, labeled_real_data, labels)

viz.generate_plots('real_data_for_adaptation', labeled_real_data, labels, results)

run.log_metric('Correct_labels', results['WORD_Corr'])
run.log_artifact('final_model', '../models/hmm/' + str(model), overwrite=True)

  Using baseclass macro "global" from file ../models/classes/global
Attached 12 XFormInfo structures
Attached 12 RegAcc structures
Pruning-Off
Generating transform 2 (1)
  Using baseclass macro "global" from file ../models/classes/global
Estimated XForm 1 using 37837.023666 observations

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C ../src/models/HTK/misc/config -H ../models/hmm/7/macros -H ../models/hmm/7/hmmdefs -A -J ../models/classes/ -J ../models/xforms mllr1 -h ../data/interim/2017.train_D_A.real/%* -k -w ../src/models/HTK/misc/wdnet -S ../data/interim/2017.train_D_A.real.scp -i ../data/interim/results.mlf ../src/models/HTK/misc/dict ../src/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Tue Sep  8 15:18:57 2020
  Ref : ../data/raw/dmps/manual_labels.mlf
  Rec : ../data/interim/results.mlf
------------------------ Overall Results --------------------------
SENT: %Correct=33.33 [H=90, S=180, N=270]
W

In [13]:
result = hmm.predict(model, unlabeled_real_data)
viz.generate_plots('labeled_real_data', unlabeled_real_data, results)

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C ../src/models/HTK/misc/config -H ../models/hmm/7/macros -H ../models/hmm/7/hmmdefs -A -J ../models/classes/ -J ../models/xforms mllr1 -h ../data/interim/2015.test_D_A.real/%* -k -w ../src/models/HTK/misc/wdnet -S ../data/interim/2015.test_D_A.real.scp -i ../data/interim/results.mlf ../src/models/HTK/misc/dict ../src/models/HTK/misc/monophones 

